In [1]:
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series

In [2]:
from math import sqrt,fabs

def svan(s, t, po):
    """ s - salinity;
        t - temperature;
        Po - is a pressure in decibars (approx. equal to meters of depth)
        svan returns sigma (seawater density, where 1000 kg/m3
        has been subtracted)"""
    
    r3500 = 1028.1063
    r4 = 4.8314E-4
    dr350 = 28.106331

    p=po/10.
    sr= sqrt(fabs(s))

    r1= ((((6.536332E-9*t-1.120083E-6)*t+1.001685E-4)*
        t-9.095290E-3)*t+6.793952E-2)*t-28.263737
    r2= (((5.3875E-9*t-8.2467E-7)*t+7.6438E-5)*t-4.0899E-3)*t+8.24493E-1
    r3= (-1.6546E-6*t+1.0227E-4)*t-5.72466E-3

    sig=(r4*s + r3*sr + r2)*s +r1

    v350p=1.0/r3500
    sva=-sig*v350p/(r3500+sig)
    sigma= sig + dr350

    if p != 0.0: 
        e = (9.1697E-10*t+2.0816E-8)*t-9.9348E-7
        bw = (5.2787E-8*t-6.12293E-6)*t+3.47718E-5
        b = bw + e*s

        d= 1.91075E-4
        c = (-1.6078E-6*t-1.0981E-5)*t+2.2838E-3
        aw = ((-5.77905E-7*t+1.16092E-4)*t+1.43713E-3)*t-0.1194975
        a = (d*sr + c)*s + aw

        b1 = (-5.3009E-4*t+1.6483E-2)*t+7.944E-2
        a1 = ((-6.1670E-5*t+1.09987E-2)*t-0.603459)*t+54.6746
        kw = (((-5.155288E-5*t+1.360477E-2)*t-2.327105)*t + 148.4206)*t-1930.06
        ko = (b1*sr + a1)*s + kw

        dk = (b*p+a)*p+ko
        k35 = (5.03217E-5*p+3.359406)*p+21582.27
        gam=p/k35
        pk=1.0-gam
        sva = sva * pk + (v350p+sva)*p*dk/(k35*(k35+dk))

        v350p= v350p*pk
        dr35p=gam/v350p
        dvan= sva/(v350p*(v350p+sva))
        sigma = dr350 + dr35p -dvan
        return sigma
    else:
        return sigma
    
def gargett(sigma, dz):
    """Returns array of approximated diffusivity
       kz, sigma - equal size arrays of values
       dz = constant value"""
    
    sigma_prev = sigma[:-1]
    sigma_next = sigma[1:]
    
    kz = [5e-7 / sqrt(9.81 / (1000 + (sigma_p + sigma_n) / 2)
                           * max(1e-8, (fabs(sigma_n - sigma_p) / dz)
                                )
                     )
          for sigma_p, sigma_n in zip(sigma_prev, sigma_next)]
    kz.append(kz[-1])
    
    return kz

def surface_radiation(day, latitude):
    """Returns surface radiation which is based on a day and 
       a current latitude"""
    
    #Theoretical maximum 24-hr average surface downwelling
    #shortwave irradiance in air [W/m2] (default = 180 W/m2)
    #http://www.soda-pro.com
    #This should include that effect of average cloud cover (local)
    Io = 180
    part_of_active_radiation = 0.5
    
    #Compute surface shortwave downwelling irradiance [W m^-2, 24-hr average]
    #Solar declination in degrees
    decl = 23.5 * np.sin(2 * np.pi * (day - 81) / 365)
    
    #This is the approximation used in Yakushev and Sorensen (2013) for OXYDEP
    surface_radiative_flux = max(0, Io * np.cos((latitude - decl) * np.pi / 180))
    surface_radiative_flux = part_of_active_radiation * surface_radiative_flux
    
    return surface_radiative_flux

In [3]:
def read_schism(file_name, number_of_layers, actual_station):
    """read a Schism output file and make a python dictionary for the actual station"""
    
    station = actual_station - 1
    vault = {'Time': [], 'Data': []}
    storage = {'Time', 'Data'}
    with open(file_name, encoding = 'utf-8') as data:
        for line in data:
            values = [float(value) for value in line.split()]
            time = values[0]
            storage = {'Time': [time for _ in range(number_of_layers)],
                       'Data': values[1 + number_of_layers * station:
                                      1 + number_of_layers * station + number_of_layers]}
            vault['Time'].extend(storage['Time'])
            vault['Data'].extend(storage['Data'])
    return vault


def adddepth(group):
    group['Depth'] = [item for item in range(0, 42, 2)]
    return group

In [4]:
temperature = DataFrame(read_schism('temp_Q20x20_s61_134.18', 21, 36))
salinity    = DataFrame(read_schism('salt_Q20x20_s61_134.18', 21, 36))
data = temperature.copy()
data = data.rename(columns={'Data':'Temperature'})
data[['Salinity']] = salinity[['Data']]
data = data.groupby('Time').apply(adddepth)

In [5]:
snapface = DataFrame({
    'Depth': [item for item in range(0, 37, 2)],
    'Time': [0 for item in range(0, 37, 2)],
    'Temperature': [0 for item in range(0, 37, 2)],
    'Salinity': [0 for item in range(0, 37, 2)],
    'Sigma': [0 for item in range(0, 37, 2)],
    'Kz': [0 for item in range(0, 37, 2)]
})

snapcntr = DataFrame({
    'Depth': [item for item in range(1, 36, 2)],
    'Time': [0 for item in range(1, 36, 2)],
    'Temperature': [0 for item in range(1, 36, 2)],
    'Salinity': [0 for item in range(1, 36, 2)],
    'Sigma': [0 for item in range(1, 36, 2)]
})

In [12]:
from netCDF4 import Dataset

rootgrp = Dataset("North_sea_36.nc", "w", format='NETCDF3_CLASSIC')

levelface = rootgrp.createDimension("levelface", 19)
levelcntr = rootgrp.createDimension("levelcntr", 18)
time = rootgrp.createDimension("time", None)

times = rootgrp.createVariable("time","f8",("time",))
times.units = "seconds since 2015-01-01 00:00:00.0"
times.calendar = "gregorian"

par = rootgrp.createVariable("par","f4",("time",))
par.units = "mol photons m−2 day−1"

hice = rootgrp.createVariable("hice","f4",("time",))
snowthick = rootgrp.createVariable("snowthick","f4",("time",))
icesurft = rootgrp.createVariable("icesurft","f4",("time",))

levelsface = rootgrp.createVariable("levelface","i4",("levelface",))
levelsface.units = "m"
levelscntr = rootgrp.createVariable("levelcntr","i4",("levelcntr",))
levelscntr.units = "m"
temperature = rootgrp.createVariable("temperature","f8",("time","levelcntr",))
temperature.units = "C degrees"
salinity = rootgrp.createVariable("salinity","f8",("time","levelcntr",))
salinity.units = "psu"
sigma = rootgrp.createVariable("sigma","f8",("time","levelcntr",))
sigma.units = "kg m-1"
turbulence = rootgrp.createVariable("turbulence","f8",("time","levelface",))
turbulence.units = "m2 s"

levelsface[:] = [item for item in range(36, -1, -2)]
levelscntr[:] = [item for item in range(35, 0, -2)]

In [13]:
i = 0
for time, subset in data.groupby('Time'):
    if time == int(time):
        snapface.Time = [time for i in range(0, 37, 2)]
        snapcntr.Time = [time for i in range(1, 36, 2)]
        snapface.Temperature = [np.interp(x, subset.Depth.values[:-2], subset.Temperature.values[:-2])
                                for x in snapface.Depth.values]
        snapcntr.Temperature = [np.interp(x, subset.Depth.values[:-2], subset.Temperature.values[:-2])
                                for x in snapcntr.Depth.values]
        snapface.Salinity = [np.interp(x, subset.Depth.values[:-2], subset.Salinity.values[:-2])
                             for x in snapface.Depth.values]
        snapcntr.Salinity = [np.interp(x, subset.Depth.values[:-2], subset.Salinity.values[:-2])
                             for x in snapcntr.Depth.values]
        snapface.Sigma = [svan(salinity, temperature, pressure) 
                          for salinity, temperature, pressure in zip(snapface.Salinity.values, 
                                                                     snapface.Temperature.values, 
                                                                     snapface.Depth.values)] 
        snapcntr.Sigma = [svan(salinity, temperature, pressure) 
                          for salinity, temperature, pressure in zip(snapcntr.Salinity.values, 
                                                                     snapcntr.Temperature.values, 
                                                                     snapcntr.Depth.values)]
        snapface.Kz = gargett(snapface.Sigma, 2)
    
        times[i] = (time - 300) * 24 * 60 * 60
        par[i] = surface_radiation(int(time - 300.25), 54.29)
        hice[i] = 0
        snowthick[i] = 0
        icesurft[i] = 0
    
        temperature[i,:] = snapcntr.Temperature.values[::-1]
        salinity[i,:] = snapcntr.Salinity.values[::-1]
        sigma[i,:] = snapcntr.Sigma.values[::-1]
        turbulence[i,:] = snapface.Kz.values[::-1]
        i += 1
    
rootgrp.close()